In [1]:
import json
from collections.abc import Sequence
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wandb

from e2e_sae.log import logger
from e2e_sae.plotting import plot_facet, plot_per_layer_metric
from e2e_sae.scripts.analysis.plot_settings import (
    SIMILAR_CE_RUNS,
    SIMILAR_RUN_INFO,
    STYLE_MAP,
)
from e2e_sae.scripts.analysis.utils import create_run_df, get_df_gpt2
from e2e_sae.scripts.analysis.plot_performance import format_two_axes

/home/raymond/miniconda3/envs/mi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
project = "raymondl/tinystories-1m_play"
runs = api.runs(project, filters={"tags": "local"})
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)

plot_facet(
    df=df,
    xs=["L0", "sparsity_coeff"],
    y="CELossIncrease",
    facet_by="layer",
    facet_vals=[4],
    line_by="run_type",
    xlabels=["L0", "Sparsity Coefficient"],
    ylabel="CE Loss Increase",
    legend_title="SAE Type",
    axis_formatter=partial(format_two_axes, better_labels=True),
    out_file="plots/l0_vs_ce_loss_layer_4.png",
    xlims=[{4: (0, 300)}, {4: (0, None)}],
    ylim={4: (0.6, 0)},
    styles=STYLE_MAP,
)

Processing runs: 100%|██████████| 21/21 [00:00<00:00, 20695.58it/s]


Run local_bayesian_bayesian_seed-0_lpcoeff-6e-05_in-to-out-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping


2025-04-29 14:46:54 - INFO - Saved to plots/l0_vs_ce_loss_layer_4.png


In [6]:
df['names']

KeyError: 'names'